In [1]:
import numpy as np
import pandas as pd
from random import randrange

In [2]:
df = pd.read_csv('..\\Datasets\\pulsar_data.csv')

In [3]:
df.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,121.156250,48.372971,0.375485,-0.013165,3.168896,18.399367,7.449874,65.159298,0.0
1,76.968750,36.175557,0.712898,3.388719,2.399666,17.570997,9.414652,102.722975,0.0
2,130.585938,53.229534,0.133408,-0.297242,2.743311,22.362553,8.508364,74.031324,0.0
3,156.398438,48.865942,-0.215989,-0.171294,17.471572,NaN,2.958066,7.197842,0.0
4,84.804688,36.117659,0.825013,3.274125,2.790134,20.618009,8.405008,76.291128,0.0


In [4]:
df = df.dropna()
df.isna().sum()
df.rename(columns={' Mean of the integrated profile': '1', ' Standard deviation of the integrated profile': '2',
                  ' Excess kurtosis of the integrated profile': '3', ' Skewness of the integrated profile': '4', ' Mean of the DM-SNR curve': '5', ' Standard deviation of the DM-SNR curve': '6',
                  ' Excess kurtosis of the DM-SNR curve': '7', ' Skewness of the DM-SNR curve': '8'}, inplace=True)

In [21]:
test = df[:100]

# Decision Tree

Classification and Regression Trees (CART)  is an acronym introduced by Leo Breiman to refer to Decision Tree algorithms that can be used for classification or regression predictive modeling problems. The representation of the CART model is a binary tree, with a node representing a single input variable (X) and a split point on that variable. The leaf nodes (also called terminal nodes) of the tree contain an output variable (y) which is used to make a prediction.

To create the binary tree, involves dividing up the input space using a greedy approach called recursive binary splitting. For that, we simply line up all the values and try different split points, testing them using a cost function. The split with the best cost (lowest cost because we minimize cost) is selected. All input variables and all possible split points are evaluated and chosen in a greedy manner based on the cost function. Splitting will continue until nodes contain a minimum number of training examples or a maximum tree depth is reached.

#  Gini Index

The Gini index will be the cost function used in this project. A split in the node will take in one input attribute and its corresponding split point. The Gini dcore will then give an idea of how good that split was by how mixed the classes are in the two resulting groups created by the split. A perfect separation results in a Gini score of 0, whereas the worst case split result in a Gini score of 0.5 (For binary classification) showing that there is equal distribution of elements across some classes. The Gini index can be calculated as:
    G =p(1) ∗ (1−p(1)) + p(2) ∗ (1−p(2)) or 1 - P(1)^2 + P(2)^2
where p(1) and p(2) is the probability  of chosing that class in a node. To find the Gini impurity 
of the whole split we use:
    N(1)/n * G(1) + N(2)/n * G(2)
where N(1) and N(2) are the number of elements in that node, n is total number of elements between the two nodes, and G(1) and G(2) are the corresponding Gini scores for the nodes.

In [6]:
def Gini(split):
    # split is list of children nodes split from 1 parent 
    total_n = sum([len(node) for node in split])
    total_gini = 0
    
    for node in split:
        n = len(node)
        
        if n ==0:
            continue
            
        score = 0
        
        for class_val in node.target_class.unique():
            p = node['target_class'].value_counts()[class_val] / n
            score += p**2
        
        total_gini += (1.0 - score) * (n / total_n)
        
    return total_gini

In [7]:
def split(index, value, dataset):
    '''
    index = attribute being evaluated
    value = point of split in index
    '''
    temp = dataset[index] > value
    right = dataset[temp]
    left = dataset[~temp]    
    return (left, right)

# Evaluating Splits

Because this model is using an an exhaustive and greedy algorithm, we must check every value on each attribute as a candidate split, evaluate the cost of the split and find the best possible split we could make. To do this, we will use a dictionary to represent a given node in the tree, in it we will store the index of a chosen attribute, the value of that attribute by which to split, and the two groups of data split by the chosen split point.  

In [8]:
def find_split(dataset):
    
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    
    for name, values in dataset.items():
        
        if name == 'target_class':
            break 
            
        for value in values:
            groups = split(name, value, dataset)
            gini = Gini(groups)
            
            if gini < b_score:
                b_index, b_value, b_score, b_groups = name, value, gini, groups
                
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Building the Tree

Now that we can find the best split point for a node, we can focus on building the tree. Terminal nodes are nodes to stop growing, it can be decided through depth or the number of rows that the node is responsible for in the training dataset. Depth is the maximum number of nodes from the root node of the tree. Once a maximum depth of the tree is met, we must stop adding new nodes to prevent over fitting. Minimum Node Records, is the minimum number of training patterns that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes. The last terminal condition is if a node contains only 1 class in which case it acheived its job.

In [9]:
def to_terminal(group):
    return int(group['target_class'].mode())

In [10]:
def create_split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if left.empty or right.empty:
        node['left'] = node['right'] = to_terminal(pd.concat([left, right]))
        return
    
     # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = find_split(left)
        create_split(node['left'], max_depth, min_size, depth+1)
        
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = find_split(right)
        create_split(node['right'], max_depth, min_size, depth+1)

In [11]:
def build_tree(data, max_depth, min_size):
    root = find_split(data)
    create_split(root, max_depth, min_size, 1)
    return root

In [12]:
def predict(node, x):
    if x[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], x)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
                return predict(node['right'], x)
        else:
            return node['right']

In [13]:
# Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

In [14]:
def kfold(dataset, n_folds):
    dataset_split = list()
    copy = dataset.copy()
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        temp = list()
        while len(temp) < fold_size:
            ind = randrange(len(copy))
            temp.append(copy.iloc[ind])
            copy.drop(copy.index[ind])
        dataset_split.append(pd.DataFrame(temp))
    return dataset_split

In [15]:
def accuracy(y, yhat):
    correct = 0
    for i in range(len(yhat)):
        if y[i] == yhat[i]:
            correct += 1
    return correct / float(len(y)) * 100.0

In [16]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = kfold(dataset, n_folds)
    scores = list()
    for i in range(len(folds)):
        train_set = list(folds)
        train_set.pop(i)
        train_set = pd.concat(i for i in train_set)
        test_set = list()
        for j in range(len(folds[i])):
            copy = folds[i].iloc[j].copy()
            test_set.append(copy)
            copy[-1] = None
            predicted = algorithm(train_set, test_set, *args)
            actual = [folds[i]['target_class'].iloc[k] for k in range(len(folds[i]))]
        acc = accuracy(actual, predicted)
        scores.append(acc)
    return scores

In [22]:
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(test, decision_tree, n_folds, max_depth, min_size)

In [23]:
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [100.0, 95.0, 90.0, 95.0, 90.0]
Mean Accuracy: 94.000%


# Information Gain 

In most cases, the Gini index will be the best criterion for finding the split point of a node. However, i wanted to cover another option that could be used. Information Gain uses entropy to decide to to make a split, being calculated as:
   Information Gain = 1 - entropy
with the entropy being:
    -sum(P[i] * log(P[i]))
Where the pi is the probability of randomly picking one element of that specific class. In most cases both algorithms give the same results and since the Gini algorithm is computationally cheaper, its usally the better option. One benifit of entropy though, is that is has been proven to be stronger than the Gini Index on datasets that are heavily unbalanced.